<a href="https://colab.research.google.com/github/Siva1729-lila/Ev_vehicle_demand/blob/main/EV_ADAPTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("/content/Electric_Vehicle_Population_By_County (1).csv")
print("Dataset loaded successfully.")


Dataset loaded successfully.


In [4]:
print("Shape of the dataset:", df.shape)


Shape of the dataset: (20819, 10)


In [5]:
print(df.dtypes)


Date                                         object
County                                       object
State                                        object
Vehicle Primary Use                          object
Battery Electric Vehicles (BEVs)             object
Plug-In Hybrid Electric Vehicles (PHEVs)     object
Electric Vehicle (EV) Total                  object
Non-Electric Vehicle Total                   object
Total Vehicles                               object
Percent Electric Vehicles                   float64
dtype: object


In [6]:
print(df.isnull().sum())


Date                                         0
County                                      86
State                                       86
Vehicle Primary Use                          0
Battery Electric Vehicles (BEVs)             0
Plug-In Hybrid Electric Vehicles (PHEVs)     0
Electric Vehicle (EV) Total                  0
Non-Electric Vehicle Total                   0
Total Vehicles                               0
Percent Electric Vehicles                    0
dtype: int64


In [7]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
print("Date column converted.")


Date column converted.


In [8]:
df = df[df['Date'].notnull()]
print("Rows with valid dates:", df.shape[0])


Rows with valid dates: 20819


In [9]:
df['County'] = df['County'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')

print("Missing values filled.")
print(df[['County', 'State']].isnull().sum())


Missing values filled.
County    0
State     0
dtype: int64


In [10]:
cols_to_convert = [
    'Battery Electric Vehicles (BEVs)',
    'Plug-In Hybrid Electric Vehicles (PHEVs)',
    'Electric Vehicle (EV) Total',
    'Non-Electric Vehicle Total',
    'Total Vehicles'
]

for col in cols_to_convert:
    df[col] = df[col].astype(str).str.replace(",", "")
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Converted object columns to numeric.")


Converted object columns to numeric.


In [11]:
df = df.dropna(subset=cols_to_convert)
df.reset_index(drop=True, inplace=True)

print("Remaining rows after removing invalid numbers:", df.shape[0])


Remaining rows after removing invalid numbers: 20819


In [12]:
Q1 = df['Percent Electric Vehicles'].quantile(0.25)
Q3 = df['Percent Electric Vehicles'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df['Percent Electric Vehicles'] = np.where(
    df['Percent Electric Vehicles'] > upper_bound, upper_bound,
    np.where(df['Percent Electric Vehicles'] < lower_bound, lower_bound,
             df['Percent Electric Vehicles'])
)

print("Outliers capped successfully.")


Outliers capped successfully.


In [13]:
print("Final Data Sample:")
print(df.head())


Final Data Sample:
        Date          County State Vehicle Primary Use  \
0 2022-09-30       Riverside    CA           Passenger   
1 2022-12-31  Prince William    VA           Passenger   
2 2020-01-31          Dakota    MN           Passenger   
3 2022-06-30           Ferry    WA               Truck   
4 2021-07-31         Douglas    CO           Passenger   

   Battery Electric Vehicles (BEVs)  Plug-In Hybrid Electric Vehicles (PHEVs)  \
0                                 7                                         0   
1                                 1                                         2   
2                                 0                                         1   
3                                 0                                         0   
4                                 0                                         1   

   Electric Vehicle (EV) Total  Non-Electric Vehicle Total  Total Vehicles  \
0                            7                         460         